## Load PyEpiDAG

In [1]:
import epidag as dag
import numpy as np

## For simulation model

The **'epidag.simulation'** is used to provide tools for simulation modelling. 

### Reasons to use 'epidag.simulation'
- The parameters of my model have complicated interactions among each other.
- My model includes many random effects, so I don't want to fix the parameters in the begining.
- I don't want to rebuild my model after the parameters changed. (Intervention analysis).
- My study include Monte Carlo inference and model fitting, so I need a convienant interface.


### SimulationCore
SimulationCore is a type of object carrying all the definition of a parameter model.


### ParameterCore
ParameterCore is a type of object can be use directly in a simulation model. A ParameterCore can be generated from its parent ParameterCore or a SimulationCore. After a ParameterCore instantiated 1) the fixed nodes are assigned, 2) the random nodes are ready to be used.

### Purposed workflow

Monte Carlo simulation
1. Prepare a SimulationCore
2. For each iteration, generate a ParameterCore
3. Build simulation models with the ParamterCores
4. Collect results and analysis (summarise or fit to data)

### Example 1. Basic syntex and function, Gaussian distribution

This example shows a normal variable ($x$) with a fixed mean value ($mu_x$) and a standard deviation ($sd$) distributed from an exponential distribution

#### Step 1. generate a simulation given a Bayesian network

In [2]:
script = '''
PCore Exp1 {
   mu_x = 0
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)

#### Step 2. Instantiate a ParameterCore which 

- Hyper-parameter ($sd$) is fixed to a certain value
- A sampler of the leaf variable ($x$) is prepared

In [3]:
pc = sc.generate(nickname='exp2')
pc

sd: 0.342363, mu_x: 0

#### Step 3. Get sampler x from the ParameterCore and provide it to a simulation model.

- The sampler can be used repeatly
- You don't need to refer to its hyper-parameters while using

In [4]:
x = pc.get_sampler('x')
x

Actor x (norm(mu_x,sd)) on exp2

In [5]:
x(), np.mean([x() for _ in range(1000)])

(0.17873958982283525, 0.01728584976904414)

#### Step 4. Intervention

You can set impulse on the ParameterCore. Then, 
- The impulse will be passed down to its downstream variables
- You don't need to do anything to the sample  

In [6]:
pc.impulse({'sd': 5, 'mu_x': 100})
pc

sd: 5, mu_x: 100

In [7]:
x(), np.mean([x() for _ in range(1000)])

(98.20346292785932, 99.84582204351338)

### Example 2. Random effects, Beta-Binomial model

Example 2 is a standard example in Baysian inference. Beta-Binomial model are used to model count data ($x$) with a latent variable, probability ($prob$). 


> dag.as_simulation_core(bn, random=[...])

The option **random** defined variables which we do not want then be fixed during ParameterCore instantiation

In [8]:
script = '''
PCore BetaBinom {
    alpha = 1
    beta = 1
    n
    prob ~ beta(alpha, beta)
    x ~ binom(n, prob)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn, random=['prob'])

Since the variable $n$ is an exogenous variable, we introduce it to new ParameterCore by **exo={...}**.

To be noticed that, $prob$ has been set as a random effect, so the variable will be requested whenever new $x$ is requested

In [9]:
pc = sc.generate('exp1', exo={'n': 100})
pc

n: 100, beta: 1, alpha: 1

Again, we get a sampler $x$ for the generated ParameterCore

In [10]:
x = pc.get_sampler('x')
x()

96

**list_all** option print all variables used to sample outcome variable. You can see that $prob$ is not a fixed variable

In [11]:
x(list_all=True)

(93, {'alpha': 1, 'beta': 1, 'n': 100, 'prob': 0.9080227285011553})

In [12]:
x(list_all=True)

(29, {'alpha': 1, 'beta': 1, 'n': 100, 'prob': 0.2658249547907175})

### Example 3. Exposed variables,  Regression model

The example is a linear regression model. Dependant variable ($y$) is composed of $x*b1$ and an intercept $b0$; $var=eps^2$ is a measure of variance.

In [13]:
script = '''
PCore Regression {
    b0 = 1
    b1 = 0.5
    x = 1
    eps ~ norm(0, 1)
    y  = x*b1 + b0 + eps
    var = pow(eps, 2)
}
'''

bn = dag.bn_from_script(script)

However, $var$ is not a variable for simulation but for providing external information, so it do not need to be treated as a sampler.. 

In [14]:
sc = dag.as_simulation_core(bn)
pc = sc.generate('exp3-1')
print('Fixed variables', pc)
print('Samplers', pc.list_samplers())

Fixed variables eps: -1.0259, x: 1, b1: 0.5, b0: 1
Samplers ['var', 'y']


Use ** out ** option in generating ParameterCore can indicate the parameters should be exposed to a simulation model.

In [15]:
sc = dag.as_simulation_core(bn, out=['y'])
pc = sc.generate('exp3-2')
print('Fixed variables', pc)
print('Samplers', pc.list_samplers())

Fixed variables eps: 0.709694, var: 0.503665, x: 1, b1: 0.5, b0: 1
Samplers ['y']


### Example 4. Hierarchy, BMI model

Example 4 describes a parameter model of a BMI (body mass index) simulation model. The model include the layers of model: country, area, and people. A country have many areas; an area have many people; there are two types of people: agA and agB. 

- Each area have its amount of foodstores which can provide food to people.  
- agA and agB preform differently in the variance of BMI
- The simulation model requests the sex of agA individuals in order to model their behavour 

In [16]:
script = '''
PCore BMI {
    b0 ~ norm(12, 1)
    b1 = 0.5
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
'''

bn = dag.bn_from_script(script)

You can define hierarchies by a dictionary with 

1. parameter groups as keys and of respective parameters as values 
2. putting their children groups in the values as well

You do not need to list every variable. The variable outside the list will be optimalised in the SimulationCore

In [17]:
hie = {
    'country': ['area'],
    'area': ['b0r', 'pf', 'ps', 'foodstore', 'agA', 'agB'],
    'agA': ['bmiA', 'ageA', 'sexA'],
    'agB': ['bmiB', 'ageB']
}

sc = dag.as_simulation_core(bn, hie=hie,
                            random=['muA'],
                            out=['foodstore', 'bmiA', 'bmiB'])
sc.deep_print()

country(b0, b1, sdB, sd)
-- area(pf, b0r, ps, foodstore)
---- agB(ageB, muB, bmiB)
---- agA(sexA, ageA, bmiA, muA)


You can use a SimulationCore to generate a root ParameterCore and use the root to $breed$ children ParameterCores

In [18]:
pc = sc.generate('Taiwan', {'sd': 1})
pc_taipei = pc.breed('Taipei', 'area')
pc_taipei.breed('A1', 'agA')
pc_taipei.breed('A2', 'agA')
pc_taipei.breed('B1', 'agB')
pc_taipei.breed('B2', 'agB')

pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 12.6945)
-- Taipei (ps: 0.319429, b0r: -0.00441711, pf: 0.195384)
---- A1 (sexA: f, ageA: 20.2592)
---- A2 (sexA: m, ageA: 25.0324)
---- B1 (ageB: 30.5713, muB: 27.9757)
---- B2 (ageB: 30.7542, muB: 28.0671)


In [19]:
print('Get node from it parent')
a1 = pc_taipei.get_child('A1')
a1.print()

print('\nGet a node from root: use \'@\' to link names of ParameterCores')
a1 = pc.find_descendant('Taiwan@Taipei@A1')
a1.print()

Get node from it parent
A1 (sexA: f, ageA: 20.2592)

Get a node from root: use '@' to link names of ParameterCores
A1 (sexA: f, ageA: 20.2592)


When putting intervention in a node, the impulse will be passed to its children node automatically.

In [20]:
# see the change of muB
pc.impulse({'b0r': 10})
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 12.6945)
-- Taipei (ps: 0.319429, b0r: 10, pf: 0.195384)
---- A1 (sexA: f, ageA: 20.2592)
---- A2 (sexA: m, ageA: 25.0324)
---- B1 (ageB: 30.5713, muB: 37.9802)
---- B2 (ageB: 30.7542, muB: 38.0716)


In [21]:
pc.impulse(['b0r'])
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 12.6945)
-- Taipei (ps: 0.319429, b0r: -0.00442252, pf: 0.195384)
---- A1 (sexA: f, ageA: 20.2592)
---- A2 (sexA: m, ageA: 25.0324)
---- B1 (ageB: 30.5713, muB: 27.9757)
---- B2 (ageB: 30.7542, muB: 28.0671)


### Example 5. A simple agent-based model 

In [22]:
script = '''
PCore BMI {
    muA = 10
    bmiA ~ norm(muA, sd)
    muB ~ unif(10, 30)
    bmiB ~ norm(muB, sd)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'agA': ['muA', 'bmiA'],
    'agB': ['muB', 'bmiB']
}

dag.form_hierarchy(bn, hie).print()

root(sd)
-- agB(muB, bmiB)
-- agA(muA, bmiA)
